In [1]:
# install jupyter-dash
!pip install jupyter-dash

In [2]:
# Import required libraries
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the data into pandas dataframe
import pandas as pd

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = JupyterDash(__name__)

# Create server variable with Flask server object for use with gunicorn
#server = myspacexapp.server

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'All'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                            value = 'All',
                                            placeholder = 'Select a Launch Site here',
                                            searchable = True),
                                html.Br(),
                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                marks = {0:'0', 1000:'1000', 2000:'2000', 3000:'3000', 4000:'4000', 5000:'5000', 6000:'6000', 7000:'7000', 8000:'8000', 9000:'9000', 10000:'10000'},
                                                value = [min_payload, max_payload]),
                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))

def get_pie_chart(entered_site):
    if entered_site == 'All':
        fig = px.pie(spacex_df, values = 'class', names = 'Launch Site', title = 'Total Success Rate by Launch Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df2 = filtered_df['class'].value_counts().to_frame()
        filtered_df2.reset_index(inplace = True)
        filtered_df2.columns = ['class', 'count'] 
        fig = px.pie(filtered_df2, values = 'count', names = 'class', title = 'Success Rate for: %s' % entered_site)
        return fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
              [Input(component_id = 'site-dropdown', component_property = 'value'),
               Input(component_id = 'payload-slider', component_property = 'value')])

def get_scatter_chart(entered_site, entered_mass_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= int(entered_mass_range[0])) & (spacex_df['Payload Mass (kg)'] <= int(entered_mass_range[1]))]
    if entered_site == 'All':
        fig2 = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Correlation between Payload and Success for all sites')
        return fig2
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig2 = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Correlation between Payload and Success for: %s' % entered_site)
        return fig2

# Run the app
#app.run_server(mode='inline')
app.run_server(mode='jupyterlab')

Dash is running on http://127.0.0.1:8050/

